In [2]:
from pprint import pprint
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import numpy as np
import seaborn as sns
import pandas as pd
import arviz as az
from pathlib import Path
import nibabel as nb 
from pycimg import CImg
import matplotlib.pyplot as plt
data_dir= Path('data')



sns.reset_defaults()
#sns.set_style('whitegrid')
#sns.set_context('talk')
sns.set_context(context='talk',font_scale=0.7)

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

tfd = tfp.distributions
tfb = tfp.bijectors

dtype = tf.float64


In [9]:
class DicomDataSet(keras.utils.Sequence):
    imgs_dir = Path('/home/bohdan/Projects/cancer-detection/code/data')
     
    def __init__(self):
        # TODO 
        # rework it to contain only filenames
        (self.x, self.y) = self.load_data()
        print(self.x[0].shape)


    def load_data(self):
        idxs = [32,36,39]
        def load_with_prefix(prefix):
            filenames = [DicomDataSet.imgs_dir/f'{prefix}_00{idx}.nii.gz' for idx in idxs]
            files = [
                np.resize(nb.load(file).get_fdata(), (512,512,123))
                for file in filenames]
            

            return np.array(files)
            
        y = load_with_prefix('MASK')
        x = load_with_prefix('IMG')
        return (x, y)

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        print("Called")
        return self.x[idx], self.y[idx]


train_data = DicomDataSet()




(512, 512, 123)


In [4]:
lr = 0.001
num_epochs = 100

def run_experiment(model, loss, train_gen, val_gen):
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=lr),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()]
    )
    print("Start training the model...")
    model.fit(train_gen, epochs=num_epochs, 
        validation_data=val_gen)
    print("Model training finished.")
    _, rmse = model.evaluate(train_gen, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(val_gen, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [5]:
def create_model_inputs():
    input_shape = (512,512,123) 
    inputs = layers.Input(shape=input_shape)
    return inputs
    


In [6]:
def prior(kernel_size, bias_size, dtype):
    n = kernel_size + bias_size
    return keras.Sequential([
        tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
        )
    ])

def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    # TODO
    # posterior_model.build((n,))
    # print(posterior_model.summary())
    return posterior_model

In [7]:
def create_bnn_model():
    inputs = create_model_inputs()
    # UNet
    # Downsampling part 
    x = layers.Conv2D(
        filters=32, #TODO why 32
        kernel_size=3,
        strides=2,
        padding="same"
    )(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    prev_conv_activation = x

    for filters in [64, 128, 256]:
        print(f'Filters {filters}')
        # TODO draw
        
        x = layers.Activation("relu")(x) 
        #TODO
        # 1. How Activation layers are connected with other layers. 
        # Are they kind of some FC layer ?
        # 2. Why do we connect two relu layers one after another

        x = layers.SeparableConv2D(filters, kernel_size=3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        # We are doing BatchNormalization here, because it is a common techinuq, to deal, 
        # with changing scale in input. It just normalizes outputs of previous layers, 
        # so they are always in the same range

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, kernel_size=3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(pool_size=3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters=filters, kernel_size=1,
            strides=2, padding="same")(prev_conv_activation)
        # Collecting data from different channels
        
        # Creating a layer, that receives as input downsampled convolution
        x = layers.add(inputs=[x, residual])
        prev_conv_activation = x
    
    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(prev_conv_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        prev_conv_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    num_classes = 2
    unet_outputs = layers.Conv2D(filters=num_classes, kernel_size=3, activation="softmax", padding="same")(x)
    
    # Add bayesian element
    distribution_params = layers.Dense(units=2)(unet_outputs)
    outputs = tfp.layers.IndependentNormal(event_shape=(1, 123))(distribution_params)
    # Define the model
    model = keras.Model(inputs, outputs)
    return model



In [10]:
# Params
num_epochs = 100
train_size = 1000
batch_size = 1
# 
mse_loss = keras.losses.MeanSquaredError()
model = create_bnn_model()
print(model.summary())
run_experiment(model, mse_loss, train_data, train_data)


Filters 64
Filters 128
Filters 256
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 12 0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 256, 256, 32) 35456       input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 256, 256, 32) 128         conv2d_9[0][0]                   
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 256, 256, 32) 0           batch_normalization_15[0][0]     
_________________________________________________________

ValueError: in user code:

    /home/bohdan/Soft/anaconda/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /home/bohdan/Soft/anaconda/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/bohdan/Soft/anaconda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/bohdan/Soft/anaconda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/bohdan/Soft/anaconda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/bohdan/Soft/anaconda/lib/python3.8/site-packages/keras/engine/training.py: